# Task 2

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!pip install geopy
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Libraries imported.


In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data = BeautifulSoup(url, 'html.parser')

In [3]:
data

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1e13d41f-d551-4e4a-b61e-95f7d46fb8a8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia semi-protected p

In [4]:
table_contents=[]
table=data.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)

In [5]:
df

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Malvern, Rouge",M1B
7,North York,Don Mills North,M3B
8,East York,"Parkview Hill, Woodbine Gardens",M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


In [6]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
fixed = pd.DataFrame(columns=column_names)

li = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in li:
    fixed = fixed.append(df[df["PostalCode"]==postcode], ignore_index=True, sort=True)
    
fixed

,Borough,Neighborhood,PostalCode
0,Downtown Toronto,Central Bay Street,M5G
1,North York,Hillcrest Village,M2H
2,East York,"Parkview Hill, Woodbine Gardens",M4B
3,Scarborough,Scarborough Village,M1J
4,East York,Leaside,M4G
5,East Toronto,Studio District,M4M
6,Scarborough,"Wexford, Maryvale",M1R
7,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V
8,North York,Humber Summit,M9L
9,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",M5V


In [7]:
# read co-ordinates

In [8]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df = df.merge(coordinates, on="PostalCode", how="left")
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Task 3

In [11]:
!pip install folium
import folium

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Latitude, Longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = 'A4GIGAD2BH2HDBYRZI1BUACGQRHXGWYRNO1XJG4NMI5EB5M2' # your Foursquare ID
CLIENT_SECRET = 'KRUZDX0BURD5BFI4XVMEDRF03RJUCPP0MFSLYFMW02WIUMNU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A4GIGAD2BH2HDBYRZI1BUACGQRHXGWYRNO1XJG4NMI5EB5M2
CLIENT_SECRET:KRUZDX0BURD5BFI4XVMEDRF03RJUCPP0MFSLYFMW02WIUMNU


In [15]:
import requests
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
tdf = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [20]:
tdf.head(5)

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [21]:
print('{} venues were returned by Foursquare.'.format(tdf.shape[0]))

2121 venues were returned by Foursquare.


In [22]:
tdf.shape

(2121, 7)

In [24]:
tdf.groupby('Neighborhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16


In [25]:
print('There are {} uniques categories.'.format(len(tdf['Venue Category'].unique())))

There are 269 uniques categories.


In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(tdf[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = tdf['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
1,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
3,Bayview Village,0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
4,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.043478,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
5,Berczy Park,0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.017241,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
6,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.062500,0.0625,0.125,0.125,0.125,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00
9,Caledonia-Fairbanks,0.000000,0.0,0.000000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25


In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Pub  0.11
2    Skating Rink  0.11
3  Sandwich Place  0.11
4     Coffee Shop  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.09
1                       Bank  0.09
2                   Pharmacy  0.05
3                  Pet Store  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4               Museum  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1         Coffee Shop  0.09

                venue  freq
0                Café  0.08
1  Mexican Restaurant  0.08
2                Park  0.08
3     Thai Restaurant  0.08
4                 Bar  0.04


----Hillcrest Village----
                      venue  freq
0               Golf Course   0.2
1      Fast Food Restaurant   0.2
2  Mediterranean Restaurant   0.2
3                      Pool   0.2
4                   Dog Run   0.2


----Humber Summit----
                       venue  freq
0                Pizza Place   1.0
1                Yoga Studio   0.0
2  Middle Eastern Restaurant   0.0
3        Moroccan Restaurant   0.0
4        Monument / Landmark   0.0


----Humberlea, Emery----
                       venue  freq
0             Baseball Field   0.5
1               Food Service   0.5
2                Yoga Studio   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----Humewood-Cedarvale----
                      venue  freq
0                     Field  0.25
1              Hockey Arena  0.2

                   venue  freq
0            Coffee Shop  0.14
1           Liquor Store  0.07
2                   Bank  0.07
3            Pizza Place  0.07
4  Vietnamese Restaurant  0.07


----The Annex, North Midtown, Yorkville----
            venue  freq
0  Sandwich Place  0.15
1            Café  0.15
2     Coffee Shop  0.10
3     Pizza Place  0.05
4   Grocery Store  0.05


----The Beaches----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2                Pub   0.2
3   Asian Restaurant   0.2
4        Yoga Studio   0.0


----The Danforth  East----
                venue  freq
0                Park  0.50
1        Intersection  0.25
2   Convenience Store  0.25
3  Miscellaneous Shop  0.00
4               Motel  0.00


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.19
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4              Restaurant  0.05


----The Ki

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Pharmacy,Gym,Playground,Skating Rink,Coffee Shop,Pub,Doner Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Frozen Yogurt Shop,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Restaurant,Supermarket
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Fast Food Restaurant,Juice Bar,Spa,Butcher,Café,Restaurant


In [31]:
#Clustering

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 1, 1, 1, 3, 0, 2, 1, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2])

In [34]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

In [36]:
neighbourhoods_venues_sorted

,Cluster_Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Pharmacy,Gym,Playground,Skating Rink,Coffee Shop,Pub,Doner Restaurant,Diner
2,1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Frozen Yogurt Shop,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Restaurant,Supermarket
3,1,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,1,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Fast Food Restaurant,Juice Bar,Spa,Butcher,Café,Restaurant
5,1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Seafood Restaurant,Cheese Shop,Pharmacy,Farmers Market,Beer Bar,Belgian Restaurant
6,1,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Diner,Drugstore
7,1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Bakery,Performing Arts Venue,Pet Store,Grocery Store,Nightclub,Climbing Gym
8,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Bar,Coffee Shop,Plane,Rental Car Location,Boat or Ferry
9,2,Caledonia-Fairbanks,Park,Women's Store,Pool,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [37]:
toronto_merged = df

toronto_merged.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353


In [38]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Escape Room
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Breakfast Spot,Bank,Electronics Store,Medical Center,Restaurant,Rental Car Location,Mexican Restaurant,Donut Shop,Women's Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Escape Room
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Fried Chicken Joint,Bakery,Bank,Thai Restaurant,Athletics & Sports,Gas Station,Caribbean Restaurant,Dumpling Restaurant,Drugstore


In [39]:
toronto_merged=toronto_merged.dropna()
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

In [42]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# cluster check
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,North York,0,Clothing Store,Furniture / Home Store,Coffee Shop,Accessories Store,Boutique,Vietnamese Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [44]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Construction & Landscaping,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Escape Room
2,Scarborough,1,Intersection,Breakfast Spot,Bank,Electronics Store,Medical Center,Restaurant,Rental Car Location,Mexican Restaurant,Donut Shop,Women's Store
3,Scarborough,1,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Escape Room
4,Scarborough,1,Hakka Restaurant,Fried Chicken Joint,Bakery,Bank,Thai Restaurant,Athletics & Sports,Gas Station,Caribbean Restaurant,Dumpling Restaurant,Drugstore
5,Scarborough,1,Playground,Jewelry Store,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
6,Scarborough,1,Convenience Store,Department Store,Hobby Shop,Coffee Shop,Discount Store,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,1,Bakery,Soccer Field,Metro Station,Park,Ice Cream Shop,Intersection,Bus Line,Doner Restaurant,Donut Shop,Drugstore
8,Scarborough,1,Motel,Skating Rink,American Restaurant,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Scarborough,1,College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Diner,Drugstore
10,Scarborough,1,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Construction & Landscaping,Convenience Store,Event Space,Ethiopian Restaurant,Escape Room


In [45]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,2,Park,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
23,North York,2,Park,Convenience Store,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
25,North York,2,Food & Drink Shop,Park,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
30,North York,2,Snack Place,Airport,Park,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East YorkEast Toronto,2,Park,Intersection,Convenience Store,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
44,Central Toronto,2,Park,Bus Line,Swim School,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
48,Central Toronto,2,Tennis Court,Park,Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
50,Downtown Toronto,2,Park,Trail,Playground,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
74,York,2,Park,Women's Store,Pool,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
90,Etobicoke,2,Park,Pool,River,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [46]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,York,3,Pizza Place,Grocery Store,Bus Line,Convenience Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
96,North York,3,Pizza Place,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
100,Etobicoke,3,Pizza Place,Sandwich Place,Mobile Phone Shop,Bus Line,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant


In [47]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Fast Food Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
